### Conversion:
from fusion v2 EO.csv -> df_final_1.csv

This provides the right fields and id-codes to match the query.

These still need to be cut up to by district jsons

In [85]:
import json
import pandas as pd
import xml
import xml.etree.ElementTree as ET

In [86]:
df_org = pd.read_csv('../data/orgtbl3.csv')
df_fusion = pd.read_csv('../data/fusion v2 EO.csv')
df_districts = pd.read_csv('../data/senate_meta_v1.csv')

C:\Users\wsutt\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Select only the columns we need

In [87]:
query_cols = 'Name, Latitude, Longitude, senateId, incomeAmount, mnnSectorCode, assetAmount'
query_cols

'Name, Latitude, Longitude, senateId, incomeAmount, mnnSectorCode, assetAmount'

In [88]:
df_final = df_fusion[['Name', 'Latitude', 'Longitude', 
                      'Income Amount', 'Asset Amount',  
                      'MNN Sector', 'Senate District']]

In [89]:
df_final.dtypes

Name                object
Latitude           float64
Longitude          float64
Income Amount      float64
Asset Amount       float64
MNN Sector          object
Senate District     object
dtype: object

#### Handle NA's

In [90]:
tmp = df_final['MNN Sector']
tmp[tmp.isna()] = "NA"
df_final['mnnSector'] = tmp

sum(df_final['mnnSector'].isna())

C:\Users\wsutt\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wsutt\Anaconda3\lib\site-packages\pandas\core\generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\wsutt\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


0

In [82]:
# sum(df_final['Senate District'].isna())
# df_final[df_final['Senate District'].isna()]

#### Build Join Tables
we need to join to get a sector id-code, and senate-district id-code

note: sector id-code proceeds DESC in total prevalence, with NA added in

In [91]:
df_districts = df_districts.sort_values(by='senateId')
df_districts_v2 = df_districts[['senateId', 'name']]
df_districts_v2.head(3)

,senateId,name
32,1,First Suffolk
29,2,Second Suffolk
21,3,Hampden


In [92]:
lookup_sector = pd.DataFrame(df_final['mnnSector'].value_counts())
lookup_sector['sectorName'] = lookup_sector.index
lookup_sector.reset_index(inplace=True, drop=True)
lookup_sector['mnnSectorCode'] = lookup_sector.index + 1
lookup_sector

,mnnSector,sectorName,mnnSectorCode
0,7160,Youth & Education,1
1,6114,Human Services,2
2,3315,Other,3
3,3167,Arts & Culture,4
4,3089,Religious Organizations,5
5,2504,Philanthropy,6
6,2296,Housing,7
7,2005,Health & Healthcare,8
8,1583,NA,9
9,1385,Human Rights,10


#### Perform Merges

In [95]:
df_final = pd.merge(df_final, 
         lookup_sector, 
         how='left', 
         left_on='MNN Sector',
         right_on='sectorName'
        )

In [99]:
df_final = pd.merge(df_final, 
         df_districts_v2, 
         how='left', 
         left_on='Senate District',
         right_on='name'
        )

#### Rename + Reorder Cols to Match Query

In [108]:
query_cols
cols = query_cols.split(', ')
cols

['Name',
 'Latitude',
 'Longitude',
 'senateId',
 'incomeAmount',
 'mnnSectorCode',
 'assetAmount']

In [111]:
df_final = df_final[['Name', 'Latitude', 'Longitude', 
                  'senateId','Income Amount', 'mnnSectorCode', 
                  'Asset Amount']]

In [112]:
df_final.columns = cols

#### Analyze

In [113]:
df_final.head(3)

,Name,Latitude,Longitude,senateId,incomeAmount,mnnSectorCode,assetAmount
0,04 FOUNDATION INC,42.583460,-70.776972,20.0,NaN,6,NaN
1,1000 FRIENDS OF MASSACHUSETTS INC,42.358245,-71.062141,27.0,0.0,11,0.0
2,100TH BOMB GROUP FOUNDATION INC,42.357350,-71.054848,27.0,45965.0,4,196429.0


In [117]:
df_final['senateId'].value_counts()

27.0    2197
40.0    1522
29.0    1396
16.0    1303
28.0    1152
14.0    1122
26.0    1021
20.0    1021
1.0     1005
10.0     925
2.0      859
34.0     838
39.0     817
5.0      791
30.0     783
24.0     778
21.0     769
25.0     763
15.0     754
23.0     732
19.0     702
22.0     678
31.0     650
9.0      646
36.0     588
7.0      575
12.0     555
37.0     548
13.0     548
3.0      540
11.0     539
38.0     525
18.0     523
35.0     490
33.0     486
Name: senateId, dtype: int64

#### Output the dataframe

In [115]:
df_final.to_csv('output/df_final_1.csv', index=False)

#### Scratch work

In [ ]:
orgDataTable.addColumn('number','Youth & Education');       //1
orgDataTable.addColumn('number','Human Services');  
orgDataTable.addColumn('number','Other');  
orgDataTable.addColumn('number','Arts & Culture');  
orgDataTable.addColumn('number','Religious Organizations');  //5
orgDataTable.addColumn('number','Philanthropy');  
orgDataTable.addColumn('number','Housing');  
orgDataTable.addColumn('number','Healthcare');  
orgDataTable.addColumn('number','NA');  
orgDataTable.addColumn('number','Human Rights');            //10
orgDataTable.addColumn('number','Environment & Nature');  
orgDataTable.addColumn('number','NAN');  

In [23]:
len(df_fusion['MNN Sector'].value_counts())

10

In [21]:
query_col

'Name, Latitude, Longitude, senateId, incomeAmount, mnnSectorCode, assetAmount'

In [17]:
df_districts_v2

,senateId,name
32,1,First Suffolk
29,2,Second Suffolk
21,3,Hampden
11,4,Berkshire & Hampshire & Franklin & Hampden
9,5,Second Hampden & Hampshire
6,6,Hampshire & Franklin & Worcester
8,7,First Hampden & Hampshire
2,8,Worcester & Hampden & Hampshire & Middlesex
22,9,Worcester & Middlesex
24,10,First Worcester


In [5]:
df_fusion

,X,EIN,Name,Latitude,Longitude,Address,City,State,Zip Code,Classification,...,County,House District,Senate District,Congressional District,Years Exempt,Designation 501c,Affiliation Type,School,Org Type,MNN Status
0,1.0,454469512.0,04 FOUNDATION INC,42.583460,-70.776972,93 PLEASANT ST,MANCHESTER,MA,01944-1107,Charitable Organization,...,ESSEX,4th Essex,First Essex & Middlesex,6.0,1.249315,3,Indepedent,Priv.Op.Found.Other,Corporation,False
1,2.0,222988006.0,1000 FRIENDS OF MASSACHUSETTS INC,42.358245,-71.062141,14 BEACON STREET,BOSTON,MA,02108-3704,Charitable Organization,...,SUFFOLK,3rd Suffolk,First Suffolk & Middlesex,8.0,18.761644,3,Indepedent,Priv.Op.Found.Other,Corporation,False
2,3.0,930917084.0,100TH BOMB GROUP FOUNDATION INC,42.357350,-71.054848,2 OLIVER ST,BOSTON,MA,02109-4901,Charitable Organization,...,SUFFOLK,3rd Suffolk,First Suffolk & Middlesex,8.0,26.019178,3,Indepedent,Priv.Op.Found.Other,Corporation,False
3,4.0,42680019.0,101ST INFANTRY VETERANS ASSOCIATION INC,42.297001,-71.051882,70 VICTORY RD,BOSTON,MA,02122-2628,Post or Organization of War Veterans,...,SUFFOLK,13th Suffolk,First Suffolk,8.0,0.410845,19,Indepedent,Priv.Op.Found.Other,Corporation,False
4,5.0,42769496.0,104TH US INFANTRY VETERANS AEF ASSOCIATION INC,42.193138,-72.628099,8 COOK ST,HOLYOKE,MA,01040-3508,Post or Organization of War Veterans,...,HAMPDEN,5th Hampden,Second Hampden & Hampshire,1.0,30.432763,19,Indepedent,Priv.Op.Found.Other,Corporation,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
33962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
33963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
33964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


In [ ]:
tblname,name,poly,incomeShared,senateIdShared

In [2]:
query_col = 'Name, Latitude, Longitude, senateId, incomeAmount, mnnSectorCode, assetAmount'